# 🧪 ETL Pipeline Using Pandas and Scikit-Learn

This notebook demonstrates a simple ETL (Extract, Transform, Load) pipeline using pandas for data handling and scikit-learn for preprocessing. It excludes high-cardinality identifiers and categorical fields to avoid sparse and bloated feature sets.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

## 🟢 Step 1: Extract Data

In [2]:
file_path = 'rawdata.csv'
try:
    df = pd.read_csv(file_path)
    print("✅ Data loaded successfully")
    display(df.head())
except FileNotFoundError:
    print(f"❌ File not found: {file_path}")

✅ Data loaded successfully


,patient_id,patient_fname,patient_mname,patient_lname,gender,occupation,category,province,country,op_date,admit_type,temparature,pulse,physician_name,physician_designation,physician_specialization
0,pid1000,Sarina,Kathryn,Robertson,female,student,masters,ontario,ca,15-Sep-20,outpatient,97.1,135,patricia,family practice physician,nephrologist
1,pid1001,Teigan,Robin,Dowling,female,pilot,loco,u.s. virgin islands,us,10-May-20,outpatient,97.3,106,garcia,family practice physician,nephrologist
2,pid1001,Teigan,Robin,Dowling,female,pilot,loco,u.s. virgin islands,us,13-Mar-20,inpatient,103.5,86,mary,senior consultant,neurologist
3,pid1002,Ingrid,Nevin,Montoya,male,comedian,stageshow,south dakota,us,15-Sep-19,inpatient,96.4,100,mary,senior consultant,neurologist
4,pid1003,Tiya,Gabriel,Bains,female,comedian,stageshow,vermont,us,16-Dec-19,inpatient,94.2,81,garcia,family practice physician,nephrologist


## 🛠️ Step 2: Transform Data

In [3]:
target_column = 'target' if 'target' in df.columns else None

if target_column:
    X = df.drop(target_column, axis=1)
    y = df[target_column]
else:
    X = df.copy()
    y = None

# Drop identifier-like columns with too many unique values
identifier_cols = [col for col in X.columns if 'id' in col.lower() and X[col].nunique() > 100]
id_tracking = X[identifier_cols].reset_index(drop=True) if identifier_cols else pd.DataFrame()
X = X.drop(columns=identifier_cols, errors='ignore')

# Drop high-cardinality categorical columns (like names, emails)
high_card_cat_cols = [col for col in X.select_dtypes(include=['object', 'category']).columns if X[col].nunique() > 50]
tracking_high_card = X[high_card_cat_cols].reset_index(drop=True) if high_card_cat_cols else pd.DataFrame()
X = X.drop(columns=high_card_cat_cols, errors='ignore')

# Define numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

X_processed = preprocessor.fit_transform(X)
print("✅ Data transformed successfully")

✅ Data transformed successfully


## 💾 Step 3: Load Transformed Data

In [4]:
# Get feature names
cat_encoder = preprocessor.named_transformers_['cat'].named_steps['encoder']
cat_features = cat_encoder.get_feature_names_out(categorical_cols)
all_features = numeric_cols + list(cat_features)

processed_df = pd.DataFrame(X_processed, columns=all_features)

# Add target column if present
if y is not None:
    processed_df['target'] = y.reset_index(drop=True)

# Add tracking columns back (if any)
if not id_tracking.empty:
    processed_df = pd.concat([id_tracking, processed_df], axis=1)
if not tracking_high_card.empty:
    processed_df = pd.concat([tracking_high_card, processed_df], axis=1)

# Save to CSV
output_file = 'cleaned_data.csv'
processed_df.to_csv(output_file, index=False)
print(f"✅ Transformed data saved to {output_file}")
display(processed_df.head())

✅ Transformed data saved to cleaned_data.csv


,patient_fname,patient_mname,patient_lname,op_date,patient_id,temparature,pulse,gender_female,gender_male,occupation_agent,...,physician_designation_consultant,physician_designation_family practice physician,physician_designation_medical director,physician_designation_senior consultant,physician_specialization_cardiologist,physician_specialization_gastroenterologist,physician_specialization_immunologist,physician_specialization_nephrologist,physician_specialization_neurologist,physician_specialization_oncologist
0,Sarina,Kathryn,Robertson,15-Sep-20,pid1000,-0.441985,1.440263,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Teigan,Robin,Dowling,10-May-20,pid1001,-0.380580,-0.199214,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Teigan,Robin,Dowling,13-Mar-20,pid1001,1.522976,-1.329887,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Ingrid,Nevin,Montoya,15-Sep-19,pid1002,-0.656902,-0.538416,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Tiya,Gabriel,Bains,16-Dec-19,pid1003,-1.332357,-1.612556,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
